# Conditional Independence
**Hands‑on Notebook**


**In this notebook**
Explore **conditional independence** in chain / fork / collider.



In [1]:
import numpy as np
import pandas as pd

## Conditional Independence in **Chain / Fork / Collider**

In this section we simulate three fundamental causal structures (often called the *building blocks* of causal graphs)  
to explore how **conditional independence** behaves in each.

Reminder:
### Marginal vs Conditional Correlation

- **Marginal correlation** measures how two variables vary together *overall*, without taking any other variables into account.  
  → Example: the raw relationship between Smoking and Cancer in the population.

- **Conditional correlation** measures how two variables relate *after we fix or control for* a third variable.  
  → Example: the relationship between Smoking and Cancer **within each level of Tar exposure**.

**Key idea:**  
If two variables are correlated marginally but not conditionally, it means a third variable (a mediator or confounder) explains their association.  
Conversely, if they are independent marginally but correlated conditionally, conditioning has **opened a path** (as in collider bias).

---

### 1. Chain: A → B → C
**Interpretation:**  
B, "the mediator" transmits information or influence from A to C.  
- *Example:* Smoking → Tar in lungs → Cancer.  
- A and C are correlated because information “flows” through B.  
- **If we condition on B**, we block that path — A and C become (approximately) independent.

**Expectation:**  
- Marginal correlation: high (A and C move together).  
- Conditional correlation given B: ≈ 0 (path blocked).

---

### 2. Fork (Confounding): A ← U → C
**Interpretation:**  
U is a *common cause* (confounder) of both A and C.  
- *Example:* Genetic predisposition → Smoking and Cancer.  
- A and C appear correlated, but only because of U.  
- **If we condition on U**, we remove that shared cause and eliminate the spurious correlation.

**Expectation:**  
- Marginal correlation: high (U induces a false link).  
- Conditional correlation given U: ≈ 0 (confounding removed).

---

### 3. Collider: A → B ← C
**Interpretation:**  
B is a *common effect* (collider) of A and C.  
- *Example:*  
  - A = Smoking  
  - C = Air pollution  
  - B = Hospital admission (caused by either).  
- Normally, A and C are independent.  
- **If we condition on B** (or any descendant of B), we *create* a correlation between A and C —  
  this is known as **collider bias** or **selection bias**.

**Expectation:**  
- Marginal correlation: near 0 (A, C independent).  
- Conditional correlation given B: strong (conditioning opens the path).

---


### Conclusion
> Correlation alone can mislead: depending on the graph, conditioning can remove, reveal, or even **fabricate** relationships.
>
> Understanding which paths are open or closed (via **d-separation**) is central to causal inference.



### What the following block of code does
- Each function (`sim_chain`, `sim_fork`, `sim_collider`) simulates random data following these causal relationships.  
- We compute:
  - **Marginal correlation**: `corr(A, C)`  
  - **Conditional correlation**: `corr(A, C | middle node)` using simple binning on the conditioning variable.
- This shows how *conditioning* can either **block** or **create** associations depending on the graph structure.


In [2]:
def sim_chain(N=50_000, seed=1):
    rng = np.random.default_rng(seed)
    A = rng.normal(0,1,N)
    B = A + rng.normal(0,1,N)
    C = B + rng.normal(0,1,N)
    return pd.DataFrame(dict(A=A,B=B,C=C))

def sim_fork(N=50_000, seed=2):
    rng = np.random.default_rng(seed)
    U = rng.normal(0,1,N)
    A = U + rng.normal(0,1,N)
    C = U + rng.normal(0,1,N)
    return pd.DataFrame(dict(U=U,A=A,C=C))

def sim_collider(N=50_000, seed=3):
    rng = np.random.default_rng(seed)
    A = rng.normal(0,1,N)
    C = rng.normal(0,1,N)
    B = A + C + rng.normal(0,1,N)  # collider
    return pd.DataFrame(dict(A=A,B=B,C=C))

def corr(x,y,df):
    return np.corrcoef(df[x], df[y])[0,1]

chain = sim_chain()
fork = sim_fork()
coll = sim_collider()

print("Chain: corr(A,C)  (marginal) =", corr("A","C", chain))
print("Fork:  corr(A,C)  (marginal) =", corr("A","C", fork))
print("Collider: corr(A,C) (marginal) =", corr("A","C", coll))

# Conditioning effects
def partial_corr_xy_given_z(x,y,z,df, bins=10):
    # Approximate partial correlation by binning on z (simple classroom-friendly approach).
    df2 = df.copy()
    df2["_zb"] = pd.qcut(df2[z], q=bins, duplicates="drop")
    vals = []
    for _,grp in df2.groupby("_zb", observed=True):
        if len(grp)>5:
            vals.append(np.corrcoef(grp[x], grp[y])[0,1])
    return np.nanmean(vals)

print("\nConditioning (approx via binning):")
print("Chain: corr(A,C | B) ≈", partial_corr_xy_given_z("A","C","B", chain))
print("Fork:  corr(A,C | U) ≈", partial_corr_xy_given_z("A","C","U", fork))
print("Collider: corr(A,C | B) ≈", partial_corr_xy_given_z("A","C","B", coll))


Chain: corr(A,C)  (marginal) = 0.5801684286913067
Fork:  corr(A,C)  (marginal) = 0.4971395078854403
Collider: corr(A,C) (marginal) = -0.010648989688450418

Conditioning (approx via binning):
Chain: corr(A,C | B) ≈ 0.04719708880720996
Fork:  corr(A,C | U) ≈ 0.03878851215357193
Collider: corr(A,C | B) ≈ -0.4745972035238439


### Interpretation of Results

| Structure | Marginal Corr(A, C) | Conditional Corr(A, C \| Z) | What it shows |
|------------|--------------------:|-----------------------------:|----------------|
| **Chain** | 0.58 | 0.05 | Conditioning on the mediator **B** blocks the flow from A → B → C. |
| **Fork** | 0.50 | 0.04 | Conditioning on the confounder **U** removes the common-cause association. |
| **Collider** | −0.01 | −0.47 | Conditioning on **B** (a common effect) creates a spurious link — classic *collider bias*. |

**Summary:**  
- **Chain & Fork:** conditioning *reduces* correlation (closes the path).  
- **Collider:** conditioning *induces* correlation (opens a blocked path).


## Excersice:

 **Collider bias:** In section B, filter to the top 10% of `B` values in the collider model and compute `corr(A,C)` there.  
   - Why does this selection amplify the association?

In [3]:
# Collider model (reusing the provided function)
def sim_collider(N=50_000, seed=3):
    rng = np.random.default_rng(seed)
    A = rng.normal(0,1,N)
    C = rng.normal(0,1,N)
    B = A + C + rng.normal(0,1,N)  # collider
    return pd.DataFrame(dict(A=A,B=B,C=C))

coll = sim_collider()

# 1️⃣ Marginal correlation between A and C (no conditioning)
corr_marginal = np.corrcoef(coll["A"], coll["C"])[0,1]

# 2️⃣ Filter to top 10% of B (conditioning by selection)
threshold = np.percentile(coll["B"], 90)
coll_top10 = coll[coll["B"] >= threshold]

# 3️⃣ Correlation between A and C *after* selecting high B (collider bias)
corr_conditioned = np.corrcoef(coll_top10["A"], coll_top10["C"])[0,1]

print("Collider model:")
print(f"  Marginal corr(A, C):        {corr_marginal:.3f}")
print(f"  Corr(A, C) in top 10% of B: {corr_conditioned:.3f}")

Collider model:
  Marginal corr(A, C):        -0.011
  Corr(A, C) in top 10% of B: -0.377


In [ ]:
**Explanation — Collider Bias:**

In the collider structure A → B ← C, variables A and C are *independent* overall,
so their marginal correlation is near zero.

However, when we **condition on B** (or equivalently, select only high-B cases, such as the top 10%),
we create a *spurious negative association* between A and C.

Intuitively, if both A and C contribute to B, then among people with high B values,
those with higher A tend to have lower C (and vice versa) to keep B in the same range.

This artificial dependence is called **collider bias** or **selection bias** —
it shows that conditioning on an outcome (or its descendant) can open a path between otherwise independent causes.